In [9]:
# Import Libraries
import numpy as np
import time

In [10]:
# Check if CUDA is available and handle gracefully
try:
    import cupy as cp
    cuda_available = True
    print("CUDA is available. Using GPU for computation.")
except (ImportError, Exception) as e:
    cuda_available = False
    print(f"CUDA is not available: {str(e)}")
    print("Falling back to CPU-only computation.")

CUDA is available. Using GPU for computation.


In [11]:
#  Generate Random 3D Points
N = 10000   # Number of points
np.random.seed(42)

In [12]:
# Generate points on CPU
points_cpu = np.random.uniform(0, 100, (N, 3))
print(f"Generated {N} random 3D points.")

Generated 10000 random 3D points.


In [13]:
# CPU Distance Matrix for comparison
print("\nCalculating distance matrix on CPU...")
start_cpu = time.time()

X_cpu = points_cpu
X_squared_sum_cpu = np.sum(X_cpu**2, axis=1)
dot_product_cpu = X_cpu @ X_cpu.T
dist_matrix_cpu = np.sqrt(np.maximum(
    X_squared_sum_cpu[:, np.newaxis] + X_squared_sum_cpu[np.newaxis, :] - 2 * dot_product_cpu,
    0.0  # Ensure no negative values
))

end_cpu = time.time()
cpu_time = end_cpu - start_cpu
print(f"CPU Execution Time: {cpu_time:.2f} seconds")


Calculating distance matrix on CPU...
CPU Execution Time: 1.49 seconds


In [14]:
# GPU Distance Matrix calculation
if cuda_available:
    try:
        print("\nCalculating distance matrix on GPU...")
        # Transfer data to GPU
        points_gpu = cp.asarray(points_cpu)

        start_gpu = time.time()

        # Perform the same calculation on GPU
        X_gpu = points_gpu
        X_squared_sum_gpu = cp.sum(X_gpu**2, axis=1)
        dot_product_gpu = X_gpu @ X_gpu.T
        dist_matrix_gpu = cp.sqrt(cp.maximum(
            X_squared_sum_gpu[:, cp.newaxis] + X_squared_sum_gpu[cp.newaxis, :] - 2 * dot_product_gpu,
            0.0  # Ensure no negative values
        ))

        # Ensure all GPU operations are complete before stopping the timer
        cp.cuda.Stream.null.synchronize()

        end_gpu = time.time()
        gpu_time = end_gpu - start_gpu

        # Transfer a small sample back to CPU for verification
        dist_matrix_gpu_sample = cp.asnumpy(dist_matrix_gpu[:5, :5])

        print(f"GPU Execution Time: {gpu_time:.2f} seconds")

        # Performance comparison
        if gpu_time > 0:
            speedup = cpu_time / gpu_time
            print(f"\n Speedup = {speedup:.2f}x (GPU vs CPU)")

        # Verification
        print("\nSample CPU distances (5x5):\n", dist_matrix_cpu[:5, :5])
        print("\nSample GPU distances (5x5):\n", dist_matrix_gpu_sample)

        # Check if results are close
        if np.allclose(dist_matrix_cpu[:5, :5], dist_matrix_gpu_sample, rtol=1e-5, atol=1e-5):
            print("\nVerification: CPU and GPU results match! ✓")
        else:
            print("\nVerification: CPU and GPU results differ! ✗")

    except Exception as e:
        print(f"Error during GPU computation: {str(e)}")
        print("GPU computation failed. Using CPU results only.")
else:
    print("\nSample CPU distances (5x5):\n", dist_matrix_cpu[:5, :5])
    print("\nGPU computation was skipped due to CUDA unavailability.")


Calculating distance matrix on GPU...
GPU Execution Time: 0.07 seconds

 Speedup = 22.28x (GPU vs CPU)

Sample CPU distances (5x5):
 [[  0.         100.67500071  35.27332073 101.6360836  102.83766576]
 [100.67500071   0.          99.73350122  83.23292733  24.18558232]
 [ 35.27332073  99.73350122   0.         112.85036689 109.67857198]
 [101.6360836   83.23292733 112.85036689   0.          82.05580609]
 [102.83766576  24.18558232 109.67857198  82.05580609   0.        ]]

Sample GPU distances (5x5):
 [[0.00000000e+00 1.00675001e+02 3.52733207e+01 1.01636084e+02
  1.02837666e+02]
 [1.00675001e+02 0.00000000e+00 9.97335012e+01 8.32329273e+01
  2.41855823e+01]
 [3.52733207e+01 9.97335012e+01 0.00000000e+00 1.12850367e+02
  1.09678572e+02]
 [1.01636084e+02 8.32329273e+01 1.12850367e+02 0.00000000e+00
  8.20558061e+01]
 [1.02837666e+02 2.41855823e+01 1.09678572e+02 8.20558061e+01
  1.34869915e-06]]

Verification: CPU and GPU results match! ✓
